# <center> webscraping

### première partie du projet bloc 1

- Webscraping du site [www.allocine.fr](https://www.allocine.fr/films/)

![filtres](images/filtresSMALL.png)

## Sources :
[beautiful-soup-4](https://beautiful-soup-4.readthedocs.io/en/latest/)<br>
<br>
<br>
<br>


In [ ]:
%reset

In [2]:
import os
import re
#import httpx
import requests
import pandas as pd

from bs4 import BeautifulSoup
from IPython.display import display
from tqdm import tqdm

%config IPCompleter.greedy = True

url_site = 'https://www.allocine.fr/films/'

### On scrape tous les genres de film

In [89]:
# Scrap all categories
r = requests.get(url_site, auth=('user', 'pass'))
if r.status_code != 200:
    print("url_site error")
    
soup = BeautifulSoup(r.content, 'html.parser')
print(type(soup))

categories = []
eltCategories = soup.find('div', class_='filter-entity-section')
for elt in eltCategories.find_all('li'):
    categories.append(elt.a.text)

df_categories = pd.Series(categories)

dict_n_cat = {k:v for k, v in enumerate(categories)}
dict_cat_n = {v:k for v, k in dict_n_cat.items()}

<class 'bs4.BeautifulSoup'>


### On scrapte les films par période
[1980 - 1989] puis [1990 - 1999] ...

In [ ]:
# Scrap url of years we want to scrap the movies
# Not Working
# I cannot get the url by scraping
decades_to_scrap = ['1980 - 1989']
eltYears = eltCategories.find_next_sibling()
lstUrl = []

eltCur = eltYears.find('li')
while eltCur:
    print(eltCur)
    text = eltCur.span.text
    test = eltCur.a.get('title')
    print(test)
    if text in decades_to_scrap:
        print("**", text)
    else:
        print(eltCur.span.text)
    eltCur = eltCur.find_next_sibling()
    break

### Ou bien nous pouvons entrer les url manuellement

In [172]:
url_decades = url_site + 'decennie-1980/'
url_year = url_site + 'decennie-1980/annee-1980/'

def getNumberOfPages(elt):
    nb = 0
    while elt:
        if elt.text.isdigit():
            nb = elt.text
        elt = elt.find_next_sibling()
    return int(nb)

r = requests.get(url_year, auth=('user', 'pass'))
if r.status_code != 200:
    print("url_site error")

# We get the number of pages for this year
soup = BeautifulSoup(r.content, 'html.parser')
pagination = soup.find('div', class_='pagination-item-holder')
nb_pages = int(pagination.find_all('span')[-1].text)
assert nb_pages == getNumberOfPages(pagination.find('span'))

### On scrape chaque page de films pour une année donnée

In [175]:
for i in range(1, nb_pages+1):
    url_year_page = url_year + '?page=' + str(i)
    print(url_year_page)

https://www.allocine.fr/films/decennie-1980/annee-1980/?page=1
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=2
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=3
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=4
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=5
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=6
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=7
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=8
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=9
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=10
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=11
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=12
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=13
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=14
https://www.allocine.fr/films/decennie-1980/annee-1980/?page=15
https://www.allocine.fr/films/decennie-1980/annee

### On scrape les pays

In [ ]:
eltCountries = eltYears.find_next_sibling()
print(eltCountries)